In [1]:
import numpy as np

class Environment:
    def __init__(self, width, height):
        self.width = width
        self.height = height
        self.grid = [[0 for _ in range(width)] for _ in range(height)]  # 0 表示可飞行，1 表示障碍物

    def set_obstacle(self, x, y):
        self.grid[y][x] = 1

class ACO:
    def __init__(self, environment, num_ants, evaporation_rate, Q, alpha, beta, max_iterations):
        self.environment = environment
        self.num_ants = num_ants
        self.evaporation_rate = evaporation_rate
        self.Q = Q
        self.alpha = alpha
        self.beta = beta
        self.max_iterations = max_iterations
        self.pheromone = [[1.0 for _ in range(environment.width)] for _ in range(environment.height)]

    def find_path(self, start, goal):
        best_path = None
        for iteration in range(self.max_iterations):
            paths = []
            for _ in range(self.num_ants):
                path = self.construct_path(start, goal)
                paths.append(path)
                if best_path is None or len(path) < len(best_path):
                    best_path = path
            self.update_pheromone(paths)
        return best_path

    def construct_path(self, start, goal):
        path = []
        current_node = start
        visited = set()
        visited.add(current_node)
        while current_node != goal:
            neighbors = get_neighbors(current_node, self.environment)
            allowed_nodes = [node for node in neighbors if node not in visited and self.environment.grid[node[1]][node[0]] == 0]
            if not allowed_nodes:
                break  # 无路可走
            next_node = self.select_next_node(current_node, allowed_nodes)
            path.append(next_node)
            visited.add(next_node)
            current_node = next_node
        return path

    def select_next_node(self, current_node, allowed_nodes):
        probabilities = []
        total = 0
        for node in allowed_nodes:
            pheromone = self.pheromone[node[1]][node[0]]
            distance = 1 / manhattan_distance(current_node, node)
            probability = (pheromone ** self.alpha) * (distance ** self.beta)
            probabilities.append(probability)
            total += probability
        if total == 0:
            return allowed_nodes[np.random.choice(len(allowed_nodes))]
        probabilities = [p / total for p in probabilities]
        print(f"Current node: {current_node}, allowed nodes: {allowed_nodes}, probabilities: {probabilities}")  # Debug info
        return allowed_nodes[np.random.choice(len(allowed_nodes), p=probabilities)]


    def update_pheromone(self, paths):
        # 信息素挥发
        for y in range(self.environment.height):
            for x in range(self.environment.width):
                self.pheromone[y][x] *= (1 - self.evaporation_rate)

        # 信息素增强
        for path in paths:
            path_length = len(path)
            if path_length == 0:
                continue
            delta_pheromone = self.Q / path_length
            for (x, y) in path:
                self.pheromone[y][x] += delta_pheromone

def get_neighbors(position, environment):
    x, y = position
    neighbors = []
    for dx, dy in [(-1, 0), (1, 0), (0, -1), (0, 1)]:  # 上下左右
        nx, ny = x + dx, y + dy
        if 0 <= nx < environment.width and 0 <= ny < environment.height:
            neighbors.append((nx, ny))
    return neighbors

def manhattan_distance(a, b):
    return abs(a[0] - b[0]) + abs(a[1] - b[1])

# 示例用法
env = Environment(10, 10)
env.set_obstacle(5, 5)
aco = ACO(env, num_ants=10, evaporation_rate=0.7, Q=1.0, alpha=1.0, beta=3.0, max_iterations=100)
path = aco.find_path((0, 0), (9, 9))
print("最优路径:", path)

Current node: (0, 0), allowed nodes: [(1, 0), (0, 1)], probabilities: [0.5, 0.5]
Current node: (1, 0), allowed nodes: [(2, 0), (1, 1)], probabilities: [0.5, 0.5]
Current node: (1, 1), allowed nodes: [(0, 1), (2, 1), (1, 2)], probabilities: [0.3333333333333333, 0.3333333333333333, 0.3333333333333333]
Current node: (0, 1), allowed nodes: [(0, 2)], probabilities: [1.0]
Current node: (0, 2), allowed nodes: [(1, 2), (0, 3)], probabilities: [0.5, 0.5]
Current node: (0, 3), allowed nodes: [(1, 3), (0, 4)], probabilities: [0.5, 0.5]
Current node: (0, 4), allowed nodes: [(1, 4), (0, 5)], probabilities: [0.5, 0.5]
Current node: (0, 5), allowed nodes: [(1, 5), (0, 6)], probabilities: [0.5, 0.5]
Current node: (1, 5), allowed nodes: [(2, 5), (1, 4), (1, 6)], probabilities: [0.3333333333333333, 0.3333333333333333, 0.3333333333333333]
Current node: (1, 4), allowed nodes: [(2, 4), (1, 3)], probabilities: [0.5, 0.5]
Current node: (1, 3), allowed nodes: [(2, 3), (1, 2)], probabilities: [0.5, 0.5]
Curren